In [ ]:
import cv2
import numpy as np
import face_recognition
import winsound
import os
from scipy.spatial import distance as dist
from collections import deque
import time

# Constants
EAR_THRESHOLD = 0.21
MAR_THRESHOLD = 0.75
DROWSINESS_SCORE_THRESHOLD = 3
ALARM_DURATION = 1500
FRAME_SKIP = 2
EAR_HISTORY_LENGTH = 10
MIN_FACE_SIZE = 300
ALARM_COOLDOWN = 3
folder_path = r"C:\Users\User\Desktop\deep learn\drosiness image"

# Initialize variables
drowsiness_score = 0
alarm_triggered = False
frame_counter = 0
ear_history = deque(maxlen=EAR_HISTORY_LENGTH)
last_alarm_time = 0

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    ear_history.append(ear)
    return ear

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[2], mouth[10])
    B = dist.euclidean(mouth[4], mouth[8])
    C = dist.euclidean(mouth[3], mouth[9])
    D = dist.euclidean(mouth[0], mouth[6])
    mar = (A + B + C) / (3.0 * D)
    return mar

def get_smoothed_ear():
    return np.mean(ear_history) if ear_history else 0

def calculate_face_size(face_location):
    top, right, bottom, left = face_location
    return (bottom - top) * (right - left)

def update_drowsiness_score(avg_ear, mar, face_size, brightness):
    global drowsiness_score, alarm_triggered, last_alarm_time
    
    size_factor = max(0.8, min(1.2, face_size / MIN_FACE_SIZE))
    brightness_factor = max(0.8, min(1.2, brightness / 127))

    adjusted_ear_thresh = EAR_THRESHOLD * size_factor * brightness_factor
    adjusted_mar_thresh = MAR_THRESHOLD * size_factor * (2 - brightness_factor)

    current_time = time.time()

    if avg_ear < adjusted_ear_thresh or mar > adjusted_mar_thresh:
        increment = 3 if avg_ear < adjusted_ear_thresh * 0.8 else 1
        drowsiness_score = min(drowsiness_score + increment, DROWSINESS_SCORE_THRESHOLD * 2)
    else:
        drowsiness_score = max(drowsiness_score - 1, 0)

    if (drowsiness_score >= DROWSINESS_SCORE_THRESHOLD and 
        not alarm_triggered and 
        current_time - last_alarm_time > ALARM_COOLDOWN):
        winsound.Beep(1500, ALARM_DURATION)
        alarm_triggered = True
        last_alarm_time = current_time
    elif drowsiness_score < DROWSINESS_SCORE_THRESHOLD // 2:
        alarm_triggered = False

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FPS, 30)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_counter += 1
    if frame_counter % FRAME_SKIP != 0:
        continue

    frame = cv2.resize(frame, (800, 600))
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray_frame)

    face_locations = face_recognition.face_locations(rgb_frame, model="hog")
    landmarks = face_recognition.face_landmarks(rgb_frame, face_locations)

    if landmarks:
        for face_loc, landmark in zip(face_locations, landmarks):
            try:
                left_eye = landmark["left_eye"]
                right_eye = landmark["right_eye"]
                mouth = landmark["top_lip"] + landmark["bottom_lip"]
                face_size = calculate_face_size(face_loc)

                left_ear = eye_aspect_ratio(left_eye)
                right_ear = eye_aspect_ratio(right_eye)
                avg_ear = (left_ear + right_ear) / 2.0
                smoothed_ear = get_smoothed_ear()
                mar = mouth_aspect_ratio(mouth)

                update_drowsiness_score(smoothed_ear, mar, face_size, brightness)

                for point in left_eye + right_eye:
                    cv2.circle(frame, point, 2, (0, 255, 255), -1)
                for point in mouth:
                    cv2.circle(frame, point, 1, (255, 0, 0), -1)

                status = "AWAKE" if drowsiness_score == 0 else \
                         "WARNING" if drowsiness_score < DROWSINESS_SCORE_THRESHOLD else \
                         "DROWSY!"
                color = (0, 255, 0) if status == "AWAKE" else \
                        (0, 165, 255) if status == "WARNING" else \
                        (0, 0, 255)

                cv2.putText(frame, f"Status: {status}", (10, 30),
                            cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2)
                cv2.putText(frame, f"EAR: {smoothed_ear:.2f} (Thresh: {EAR_THRESHOLD:.2f})", (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)
                cv2.putText(frame, f"MAR: {mar:.2f} (Thresh: {MAR_THRESHOLD:.2f})", (10, 85),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)
                cv2.putText(frame, f"Score: {drowsiness_score}/{DROWSINESS_SCORE_THRESHOLD}", (10, 110),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                if alarm_triggered:
                    cv2.putText(frame, "ALERT! DROWSINESS DETECTED!", (150, 50),
                                cv2.FONT_HERSHEY_TRIPLEX, 0.9, (0, 0, 255), 2)
                    cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 10)
                    filename = f"drowsy_alert_{int(time.time())}.jpg"
                    full_path = os.path.join(folder_path, filename)
                    cv2.imwrite(full_path, frame)
            except Exception as e:
                print(f"Processing error: {e}")
    else:
        cv2.putText(frame, "NO FACE DETECTED", (250, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        drowsiness_score = max(drowsiness_score - 1, 0)

    cv2.imshow("Enhanced Drowsiness Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


SMS sent: SID SM73fffe2536b9aeba61d43734ca332fa6
